## Traveling Salesperson Problem with DWAVEQUBO (2000Q)
![](https://media.datacenterdynamics.com/media/images/dwave2000.original.png)

In [ ]:
import numpy
print(numpy.__version__)

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.manifold import MDS

http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/
<br>
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQHIh_lHSefNzbfwdeDnyiSETBwdbSo34EuZZ5QRynZnQ&s)

### ใส่ชื่อไฟล์ที่จะทำการทดลอง

In [ ]:
# open the sample file used 

file_name = 'GR17_DN-th0250.txt'
#file_name = 'FRI26_DN-th0250.txt'

### เริ่มการอ่านไฟล์ตั้งแต่บรรทัดที่ ... ถึง ... เพื่อเก็บค่า `algo_dist` (ALGORITHM_DISTANCE_MATRIX)

In [ ]:
file_path = open(file_name) 
# read the content of the file opened 
content = file_path.readlines() 
def read_algo_dist_from_file(filename):
    instance=[]
    #print(filename[0])
    first_line = filename[0]
    _, _, num_samples_str = first_line.split()
    global N_samples
    N_samples, num_samples=int(num_samples_str), int(num_samples_str)
    num_points=10
    #print(num_samples_str)


    start_line = 33
    end_line = 58
    # loop through each N-Sample
    for i, line in enumerate(filename):
        if i in range(start_line-1, end_line):
            #print("line",i)
            line=filename[i].strip()
            try:
                data=list(map(float, line.split()))
            except:
                data=list(map(float, line.split(',')))

            instance.append(data)
    return instance

algo_dist=read_algo_dist_from_file(content)

In [ ]:
print(algo_dist)

In [ ]:
n_point=len(algo_dist)
time_step=n_point

print("points = ", n_point)

### เก็บค่า `original_dist` (ORIGINAL_DISTANCE_MATRIX) เพื่อเขาไว้เป็น baseline ในการคำนวณ

In [ ]:
def read_original_dist_from_file(file_path, n_points):
    start_line = 4
    end_line = start_line + n_points
    delimiter = ','
    with open(file_path, 'r') as file: lines = file.readlines()
    
    distance_matrix=[]
    for i, line in enumerate(lines):
        if i in range(start_line, end_line):
            try:
                row=list(map(float, line.split()))
            except:
                row=list(map(float, line.split(',')))
            distance_matrix.append(row)
    
    return np.array(distance_matrix)

original_dist = read_original_dist_from_file(file_name, len(algo_dist[0]))

### Draw problem state

In [ ]:
mds=MDS(n_components=2, dissimilarity='precomputed', normalized_stress='auto', random_state=42)
coordinates=mds.fit_transform(original_dist)
coordinates-=coordinates.min(axis=0)
coordinates/=coordinates.max(axis=0)
df=pd.DataFrame(coordinates, columns=['x', 'y'])
print(len(df))
df

In [ ]:
plt.figure(figsize=(5, 5))

city_index="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
for i in range(len(df)):
    plt.scatter(df.iloc[i]['x'], df.iloc[i]['y'], c='red', s=8)
    if n_point<=26:
        plt.text(df.iloc[i]['x']+0.01, df.iloc[i]['y']+0.01, f'{city_index[i]}', fontsize=8)
    else:
        plt.text(df.iloc[i]['x']+0.01, df.iloc[i]['y']+0.01, f'{i}')

plt.title(file_name)
plt.show()

### QUBO Building

In [ ]:
class QuboPoly():
    def __init__(self, n=1024):
        self.array=np.zeros((n, n), dtype=int)
        self.constant=0
        self._size=n
    
    def add_term(self, i, j, c):
        if i>=self._size or j>=self._size:
            raise RuntimeError("Wrong index")
        self.array[i][j]+=c
        
    def add_constant(self, c):
        self.constant+=c
        
    def sum(self, p):
        if self._size != p._size:
            raise RuntimeError("Wrong polynomial size")
        self.array+=p.array
        self.constant+=p.constant
        
    def power(self):
        a=np.diag(self.array)
        self.array=np.outer(a, a) + 2*self.constant*np.diag(a) # convert back to NxN
        self.constant**=2
        
    def multiply(self, p):
        a=np.diag(self.array)
        b=np.diag(p.array)
        self.array=np.outer(a, b) + self.constant*np.diag(b) + p.constant*np.diag(a)
        self.constant*=p.constant

##### $ Obj=\sum_{t=0}^{time-1}\sum_{i=0}^{city}\sum_{j=0}^{city}x_{i,t}\cdot x_{j,t+1}\cdot d_{i,j} + \sum_{i=0}^{city}\sum_{j=0}^{city}x_{i,t_{end}}\cdot x_{j,0}\cdot d_{i,j} $ (last terms for calculating Hamiltonian cycle)

In [ ]:
def objective_function():
    qubo=QuboPoly(n_point*time_step)
    # Ex. 3 cities iterate 0 1 2-> (0,1) (0,2) (2,0)
    for t in range(time_step):
        for i in range(n_point):
            for j in range(n_point):
                #print(city_index[i], city_index[j], t)
                if t==time_step-1: 
                    qubo.add_term((i*time_step)+t, (j*time_step), algo_dist[i][j])  # last terms that the final destination reaches the original node
                else:
                    qubo.add_term((i*time_step)+t, (j*time_step)+t+1, algo_dist[i][j])
    #print(qubo.array)
    return qubo

##### $ C_{1}=\sum_{t=0}^{time}(\sum_{i=0}^{city} x_{i, t}-1)^2 $ (one-hot constraint for column)

In [ ]:
def build_one_car_each_t(alpha):
    qubo=QuboPoly(n_point*time_step)
    for t in range(time_step):
        tmp=QuboPoly(n_point*time_step)
        for i in range(n_point):
            #print((i*time_step)+t, t)
            tmp.add_term((i*time_step)+t, (i*time_step)+t, alpha)
        tmp.add_constant(-alpha)
        tmp.power()
        qubo.sum(tmp)
        #print(tmp.array)
    #print(qubo.array)
    return qubo

##### $ C_{2}=\sum_{i=0}^{city}(\sum_{t=0}^{time} x_{i, t}-1)^2 $ (one-hot constraint for row)

In [ ]:
def build_car_visit_once(alpha):
    qubo=QuboPoly(n_point*time_step)
    for i in range(n_point):
        tmp=QuboPoly(n_point*time_step)
        for t in range(time_step):
            #print((i*time_step)+t, t)
            tmp.add_term((i*time_step)+t, (i*time_step)+t, alpha)
        tmp.add_constant(-alpha)
        tmp.power()
        qubo.sum(tmp)
        #print(tmp.array)
    #print(qubo.array)
    return qubo

#### Building $Q$ matrix

In [ ]:
from time import monotonic
start_time=monotonic()  # timer

Q=QuboPoly(n_point*time_step)
Obj=objective_function()
C1=build_one_car_each_t(500) # column-alpha
C2=build_car_visit_once(500) # row-alpha
Q.sum(Obj)
Q.sum(C1)
Q.sum(C2)

print(f"Gen. Q time {monotonic() - start_time} seconds")
print("constant =", Q.constant)
#print(Q.array)
print(f"Matrix size: {Q._size} x {Q._size}")

In [ ]:
from collections import defaultdict
Q_dwave=defaultdict(int)

for i in range(Q._size):
    for j in range(Q._size):
        if Q.array[i, j]!=0:
            Q_dwave[(i, j)]=Q.array[i, j]

print("# of terms =", len(Q_dwave))
print("# of total =", Q._size**2)
print(f'Terms = {len(Q_dwave)/(Q._size**2)}%')

### Solve $x^TQx$ via D'Wave QM

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dimod import BinaryQuadraticModel
import time
bqm=BinaryQuadraticModel('BINARY')
bqm=BinaryQuadraticModel.from_qubo(Q_dwave)

cqm = ConstrainedQuadraticModel().from_bqm(bqm)

## **Quantum Annealer 2000Q**

In [ ]:
start_time=monotonic()

bqm_sampler=EmbeddingComposite(DWaveSampler())
#sampleset=bqm_sampler.sample(bqm, label=f'2000Q-GR17', num_reads=3000)
#sampleset=bqm_sampler.sample(bqm, label=f'2000Q-FRI26', num_reads=3000)
sampleset=bqm_sampler.sample(bqm, label=file_name, num_reads=3000)

print(f"D-wave time {monotonic()-start_time} seconds")

In [ ]:
print(sampleset.info)

In [ ]:
best_sample=sampleset.first.sample
best_energy=sampleset.first.energy

print("Best Sample:", best_sample)
print("Best Energy:", best_energy)

# map it back to answer
solution=np.zeros(n_point*time_step, dtype=int)
for index, var in enumerate(best_sample):
    solution[index]=int(best_sample[var])

In [ ]:
#import sys
#np.set_printoptions(threshold=sys.maxsize)
#print(solution.reshape(n_point, time_step))
solution=solution.reshape(n_point, time_step)
print(solution)

In [ ]:
def check_c1():
    for t in range(time_step):
        cnt=0
        for i in range(n_point):
            cnt+=solution[i][t]
        if cnt!=1:
            return False
    return True

def check_c2():
    for i in range(n_point):
        cnt=0
        for t in range(time_step):
            cnt+=solution[i][t]
        if cnt!=1:
            return False
    return True

In [ ]:
routes=[0]*time_step
for index, (key, val) in enumerate(best_sample.items()):
    #print(index, key, val)
    if val > 0.5:
        print(f'x{index} = {val} (city: {int(index/time_step)}, time: {index%time_step})')
        routes[index%time_step]=int(index/time_step)

# append for drawing graph with Hamiltonian cycle by hand
routes.append(routes[0])
print(routes)
print("Number of visited:", len(routes))
print("Number of unique:", len(set(routes)))
print("C1:", check_c1())
print("C2:", check_c2())

#### Visualization

In [ ]:
cost = 0
for i in range(len(routes)-1):
    cost += original_dist[routes[i]][routes[i+1]]  # DON'T FORGET TO USE THE ORIGINAL_DISTANCE_MATRIX EVERYTIME YOU MAPPED BACK TO THE ANSWER
print("Total cost:", cost)

In [ ]:
plt.figure(figsize=(7, 7))

for i in range(len(algo_dist[0])):
    plt.scatter(df.iloc[i]['x'], df.iloc[i]['y'], c='red')
    #plt.text(df.iloc[i]['x']+0.88, df.iloc[i]['y']+0.88, f'{i}')

for i in range(len(routes)-1):
    #plt.arrow(df.iloc[routes[i]]['x'], df.iloc[routes[i]]['y'], df.iloc[routes[i+1]]['x'] - df.iloc[routes[i]]['x'], df.iloc[routes[i+1]]['y'] - df.iloc[routes[i]]['y'], head_width=2, head_length=3, fc='blue', ec='blue')
    x1, y1=df.iloc[routes[i]]['x'], df.iloc[routes[i]]['y']
    x2, y2=df.iloc[routes[i+1]]['x'], df.iloc[routes[i+1]]['y']
    plt.plot([x1, x2], [y1, y2], c='blue')

    mid_x=(x1+x2)/2
    mid_y=(y1+y2)/2
    dx=(x2-x1)*0.05  # Offset along x-direction
    dy=(y2-y1)*0.05  # Offset along y-direction
    
    #plt.annotate('', xy=(mid_x + dx, mid_y + dy), xytext=(mid_x, mid_y), arrowprops=dict(arrowstyle='->', color='blue'))
    plt.annotate('', xy=(mid_x + dx, mid_y + dy), xytext=(mid_x, mid_y),
                 arrowprops=dict(arrowstyle='simple, tail_width=0.5, head_width=0.69, head_length=0.69', color='violet'))
        
    try: 
        plt.title(f"[{file_name}] LeapHybrid Total distance: {cost}")
    except:
        plt.title(f"LeapHybrid Total distance: {cost}")
plt.show()